In [1]:
print("Hello")

Hello


In [2]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

dataset_name = "glue"
task_name = "sst2"
dataset = load_dataset(dataset_name, task_name, split="train")

base_model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

Found cached dataset glue (/home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
/usr/local/lib/python3.10/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

In [4]:
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500
)

max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="sentence",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

/home/9130/.local/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Loading cached processed dataset at /home/9130/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-844e752c81825362.arrow


In [5]:
trainer.train()

/usr/local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: yifeisong7 (promptexpert). Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,4.489300
20,3.873900
30,3.220100
40,3.143600
50,3.028600
60,3.054400
70,3.055300
80,3.039000
90,3.041200
100,2.979400


TrainOutput(global_step=500, training_loss=3.0168228721618653, metrics={'train_runtime': 1085.7495, 'train_samples_per_second': 7.368, 'train_steps_per_second': 0.461, 'total_flos': 4331749677465600.0, 'train_loss': 3.0168228721618653, 'epoch': 0.12})

In [17]:
# from peft import AutoPeftModelForCausalLM

# model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map=device_map, torch_dtype=torch.bfloat16)
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative."
user_msg_1 = "for the uninitiated plays better on video with the sound"
# text = """<s>[INST] <<SYS>>
# {}
# <</SYS>>

# {} [/INST]""".format(system_prompt, user_msg_1)
text = "Classify the sentiment of the following sentence into one of the following categories: positive or negative: \nSentence: only masochistic moviegoers need apply. \nSentiment:"

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=200, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Classify the sentiment of the following sentence into one of the following categories: positive or negative: 
Sentence: only masochistic moviegoers need apply. 
Sentiment: negative

### Explanation

The sentence is negative because it is a negative statement.

### Solution

```
import java.util.*;
import java.lang.*;
import java.io.*;

class Solution {
    public static void main(String[] args) {
        Scanner sc = new Scanner(System.in);
        String s = sc.nextLine();
        System.out.println(sentiment(s));
    }

    public static String sentiment(String s) {
        if (s.length() == 0) {
            return "positive";
        }
        if (s.length() == 1) {
            return "negative";
        }
        if (s.length() == 2) {
            return "positive";
        }
        if (s.length() == 3) {
            return "negative


In [23]:
text

"<s>[INST] <<SYS>>\nYou are a helpful, respectful and honest sentiment analysis assistant. And you are supposed to classify the sentiment of the user's message into one of the following categories: positive or negative.\n<</SYS>>\n\nI am so happy today! [/INST]"